<a href="https://colab.research.google.com/github/Pulak-Deb/Kaggle_Competitions/blob/main/Concrete_Strength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# !pip install ydata_profiling

In [48]:
df=pd.read_csv('train.csv')
df.head()
test= pd.read_csv('test.csv')
test_ids= test['Row ID']

In [3]:
from ydata_profiling import ProfileReport
Report= ProfileReport(df)
Report.to_file(output_file='Report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [49]:
x = df.drop(['csMPa','Row ID'], axis=1)
y = df['csMPa']

In [50]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.model_selection import train_test_split
Preprocess = MinMaxScaler()
y = Preprocess.fit_transform(y.values.reshape(-1,1))
y = pd.DataFrame(y, columns=['csMPa'])
x_train,x_test,y_train,y_test= train_test_split(x,y, test_size=0.2)
x_train
test.drop('Row ID', axis=1, inplace=True)

In [51]:
numeric_columns = x.select_dtypes(include=['number']).columns

In [52]:
Preprocessing = make_column_transformer(
    (MinMaxScaler(),numeric_columns),
    #(PowerTransformer(),numeric_columns)
)
x_train_scaled= Preprocessing.fit_transform(x_train)
x_test_scaled= Preprocessing.transform(x_test)
x_train_scaled= pd.DataFrame(x_train_scaled,columns=x_train.columns )
x_test_scaled= pd.DataFrame(x_test_scaled,columns=x_test.columns )
x_train_scaled
test_scaled= Preprocessing.transform(test)
test_scaled= pd.DataFrame(test_scaled,columns=test.columns )
test_scaled

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age
0,0.149315,0.361436,0.6430,0.428914,0.242236,0.615962,0.382840,0.035714
1,0.350457,0.473845,0.0000,0.510383,0.000000,0.676868,0.326894,0.074176
2,0.330594,0.000000,0.6255,0.171725,0.372671,0.857486,0.519067,0.005495
3,0.494977,0.591263,0.0000,0.270767,0.444099,0.153315,0.718515,0.074176
4,0.266895,0.000000,0.6205,0.293131,0.350932,0.833183,0.504014,0.151099
...,...,...,...,...,...,...,...,...
304,0.737443,0.295771,0.0000,0.253195,0.512422,0.153315,0.735324,0.151099
305,0.399772,0.000000,0.4870,0.309904,0.366460,0.518752,0.706473,0.074176
306,0.129224,0.414302,0.5800,0.425719,0.465839,0.456946,0.315354,0.074176
307,0.502283,0.000000,0.5800,0.592652,0.310559,0.051005,0.549423,0.074176


In [53]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop, SGD
from kerastuner.tuners import RandomSearch
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization


In [38]:
# Define a function to build the model
def build_model(hp):
    model = Sequential()

    # Input layer
    model.add(Dense(
        units=hp.Int('input_units', min_value=16, max_value=128, step=16),
        activation=hp.Choice('input_activation', ['relu', 'tanh', 'sigmoid']),
        input_dim=8))  # Replace with the number of input features

    # Hidden layers
    for i in range(hp.Int('num_hidden_layers', 1, 3)):  # Tune number of hidden layers
        model.add(Dense(
            units=hp.Int(f'layer_{i+1}_units', min_value=16, max_value=128, step=16),
            activation=hp.Choice(f'layer_{i+1}_activation', ['relu', 'tanh', 'sigmoid'])
        ))

    # Output layer
    model.add(Dense(
        units=1,  # Adjust for multi-class classification (e.g., units=num_classes)
        activation='linear'))  # Adjust for your task type

    # Compile the model
    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd']),
                  loss='mean_squared_error',  # Change for regression or multi-class classification
                  metrics=['accuracy'])

    return model

# Initialize KerasTuner RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',  # Optimize for validation accuracy
    max_trials=5,  # Number of models to test
    executions_per_trial=2,  # Average results over these runs
    directory='my_dir',
    project_name='ann_tuning_units_activation1'
)


# Perform the search
tuner.search(x_train_scaled, y_train, epochs=10,validation_data=(x_test_scaled,y_test), verbose=1)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print(f"\nBest Hyperparameters Found:")
print(f" - Input Units: {best_hps.get('input_units')}, Activation: {best_hps.get('input_activation')}")
print(f" - Number of Hidden Layers: {best_hps.get('num_hidden_layers')}")
for i in range(best_hps.get('num_hidden_layers')):
    print(f"   Layer {i+1} Units: {best_hps.get(f'layer_{i+1}_units')}, Activation: {best_hps.get(f'layer_{i+1}_activation')}")
print(f" - Optimizer: {best_hps.get('optimizer')}")




Trial 5 Complete [00h 00m 09s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.0
Total elapsed time: 00h 00m 37s

Best Hyperparameters Found:
 - Input Units: 96, Activation: relu
 - Number of Hidden Layers: 1
   Layer 1 Units: 32, Activation: relu
 - Optimizer: sgd


In [39]:
# # Build the best model
# best_model = tuner.hypermodel.build(best_hps)

# # Train the best model
# history = best_model.fit(x_train_scaled, y_train, epochs=100,validation_data=(x_test_scaled,y_test), verbose=1)

# Evaluate the model
# loss, accuracy = best_model.evaluate(X, y, verbose=0)
# print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0021 - loss: 0.1564 - val_accuracy: 0.0000e+00 - val_loss: 0.0649
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0018 - loss: 0.0531 - val_accuracy: 0.0000e+00 - val_loss: 0.0506
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 4.9194e-04 - loss: 0.0442 - val_accuracy: 0.0000e+00 - val_loss: 0.0472
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.8275e-04 - loss: 0.0435 - val_accuracy: 0.0000e+00 - val_loss: 0.0441
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0021 - loss: 0.0411 - val_accuracy: 0.0000e+00 - val_loss: 0.0418
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0012 - loss: 0.0354 - val_accuracy: 0.0000e+00 - val_loss: 0.0396
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 7.3594e-04 - loss: 0.0383 - val_accuracy: 0.0000e+00 - val_loss: 0.0377
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0010 - loss: 0.03

In [60]:
model= Sequential()
model.add(Dense(96, activation='relu', input_dim=x_train.shape[1]))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(x_train_scaled, y_train, epochs=200, batch_size=32 , validation_data=(x_test_scaled, y_test), verbose=1)


Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 1.7327 - val_loss: 0.0856
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.1949 - val_loss: 0.0852
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8617 - val_loss: 0.0914
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.7399 - val_loss: 0.0998
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6166 - val_loss: 0.1197
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5694 - val_loss: 0.1384
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5423 - val_loss: 0.1564
Epoch 8/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3917 - val_loss: 0.1610
Epoch 9/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3557 - val_loss: 0.1656
Epoch 10/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3479 - val_loss: 0.1718
Epoch 11/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3578 - val_loss: 0.1707
Epoch 12/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3010 - val_l

In [61]:
y_pred = model.predict(x_test_scaled)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [62]:
from sklearn.metrics import r2_score,mean_squared_error
print('r2_score: ',r2_score(y_test, y_pred))
print('mean_squared_error: ',mean_squared_error(y_test, y_pred))

r2_score:  0.8442258271808408
mean_squared_error:  0.008127769503295454


In [63]:
submission_pred= Preprocess.inverse_transform(model.predict(test_scaled))
submission_pred

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[24.714577 ],
       [29.807125 ],
       [29.36717  ],
       [52.27447  ],
       [47.051388 ],
       [24.170881 ],
       [52.132893 ],
       [22.617533 ],
       [50.512424 ],
       [30.433388 ],
       [34.58687  ],
       [37.28373  ],
       [16.985739 ],
       [34.456654 ],
       [19.627285 ],
       [68.88977  ],
       [63.71141  ],
       [46.187065 ],
       [22.642956 ],
       [17.097748 ],
       [46.51789  ],
       [64.381805 ],
       [28.983929 ],
       [29.592113 ],
       [40.862812 ],
       [42.951168 ],
       [26.815765 ],
       [18.099539 ],
       [35.169228 ],
       [32.532635 ],
       [20.81219  ],
       [29.728792 ],
       [34.69706  ],
       [50.5167   ],
       [39.072487 ],
       [39.835    ],
       [39.618504 ],
       [25.200943 ],
       [22.765863 ],
       [39.20951  ],
       [45.002007 ],
       [41.699696 ],
       [14.290811 ],
       [26.668613 ],
       [25.100393 ],
       [43.578552 ],
       [18.34791  ],
       [24.85

In [65]:
submit= pd.DataFrame({'Row ID':test_ids.values,'csMPa':submission_pred.flatten()})
submit.to_csv('mysubmission.csv',index=False)